# Revit <> Python Schedule Aggregation

* This doc is an example of combining all spreadsheets from different schedules into a unified spreadsheet
* This process is called data wrangling / data aggregation

* Please change the directories in the python files in order to make it accessible to your machine

_bruns-ai_ (2023)

In [184]:
# used for dataframe aggreagation
import pandas as pd 

# Used for file searching & Directories
import os
import glob 

#### Change Directory to Revit Files

In [2]:
directory_name = 'C:\\Users\\Shaun Gan\\Desktop\\bruns-ai\\revit'
os.chdir(directory_name)

#### Obtain all Module Folders

In [176]:
department = 'Architectural'
modules_architectural = glob.glob(directory_name + "\\{d}\\Sample of CSV File\\*".format(d = department))
modules_architectural

['C:\\Users\\Shaun Gan\\Desktop\\bruns-ai\\revit\\Architectural\\Sample of CSV File\\Module A1',
 'C:\\Users\\Shaun Gan\\Desktop\\bruns-ai\\revit\\Architectural\\Sample of CSV File\\Module A10',
 'C:\\Users\\Shaun Gan\\Desktop\\bruns-ai\\revit\\Architectural\\Sample of CSV File\\Module A11',
 'C:\\Users\\Shaun Gan\\Desktop\\bruns-ai\\revit\\Architectural\\Sample of CSV File\\Module A12',
 'C:\\Users\\Shaun Gan\\Desktop\\bruns-ai\\revit\\Architectural\\Sample of CSV File\\Module A13',
 'C:\\Users\\Shaun Gan\\Desktop\\bruns-ai\\revit\\Architectural\\Sample of CSV File\\Module A14',
 'C:\\Users\\Shaun Gan\\Desktop\\bruns-ai\\revit\\Architectural\\Sample of CSV File\\Module A15',
 'C:\\Users\\Shaun Gan\\Desktop\\bruns-ai\\revit\\Architectural\\Sample of CSV File\\Module A2',
 'C:\\Users\\Shaun Gan\\Desktop\\bruns-ai\\revit\\Architectural\\Sample of CSV File\\Module A3',
 'C:\\Users\\Shaun Gan\\Desktop\\bruns-ai\\revit\\Architectural\\Sample of CSV File\\Module A3b',
 'C:\\Users\\Shaun Gan\

#### Function to Read All Schdules in One Module

In [178]:
def read_module(schedules):
    """
    Function to read schedules within 1 module folder, 
        e.g. (T-BSP-ARC-OTR-ALL-A9-R00_ModDoorSchedule.csv)
    
    schedules: list
        List of Schedule Directories found within the module folder
    """
    
    # Overall Dataframe that will be exported, starts empty
    overall_df = pd.DataFrame()
    
    for s in schedules:
        
        # Read csv file, delimiter is '|', in future it should be ','
        tmp_df = pd.read_csv(s,delimiter = '|',skiprows=[0,2])
        
        # Obtain File name, module number and schedule name
        file_info = s.split('\\')[-1].split('-')
        name_module = file_info[-2]
        name_schedule = file_info[-1].split('.')[0]

        # Insert names of module back into dataframe
        tmp_df.insert(loc=0, column='Module', value=[name_module]*len(tmp_df))
        tmp_df.insert(loc=1, column='Schedule', value=[name_schedule]*len(tmp_df))
        
        # Combine individual schedule csvs into a master dataframe
        overall_df = overall_df.append(tmp_df)

    print("Reading Complete - Module {} : Found {} Schedules".format(name_module,len(schedules)))
    
    return overall_df

#### Read all schedules in every module

In [188]:
output_df = pd.DataFrame()

for module in modules_architectural:
    
    # Set Directory
    schedule_directory = module + "\\*.csv"
    
    # Obtain all schedules within module directory
    schedules = glob.glob(schedule_directory)
    
    # Combine combined schedule dataframes returned from read_module function into one dataframe
    output_df = output_df.append(read_module(schedules))
    
output_df = output_df.reset_index(drop = True)

Reading Complete - Module A1 : Found 3 Schedules
Reading Complete - Module A10 : Found 3 Schedules
Reading Complete - Module A11 : Found 4 Schedules
Reading Complete - Module A12 : Found 4 Schedules
Reading Complete - Module A13 : Found 2 Schedules
Reading Complete - Module A14 : Found 2 Schedules
Reading Complete - Module A15 : Found 2 Schedules
Reading Complete - Module A2 : Found 2 Schedules
Reading Complete - Module A3 : Found 3 Schedules
Reading Complete - Module A3b : Found 3 Schedules
Reading Complete - Module A4 : Found 3 Schedules
Reading Complete - Module A5 : Found 3 Schedules
Reading Complete - Module A7 : Found 3 Schedules
Reading Complete - Module A7a : Found 3 Schedules
Reading Complete - Module A8 : Found 2 Schedules
Reading Complete - Module A9 : Found 3 Schedules


#### Save File

In [189]:
output_file_name = 'Architectural_Schedules.csv'
save_directory = "C:\\Users\\Shaun Gan\\Desktop\\bruns-ai\\revit\\{}".format(output_file_name)

output_df.to_csv(output_file_name)

In [190]:
# Preview of dataframe
output_df

Module                     Schedule  \
0       A1          R00_ModDoorSchedule   
1       A1          R00_ModDoorSchedule   
2       A1          R00_ModDoorSchedule   
3       A1          R00_ModDoorSchedule   
4       A1   R00_ModPartMaterialTakeoff   
..     ...                          ...   
238     A9  R00_ModPartMaterialSchedule   
239     A9  R00_ModPartMaterialSchedule   
240     A9  R00_ModPartMaterialSchedule   
241     A9        R00_ModWindowSchedule   
242     A9        R00_ModWindowSchedule   

                                       Family and Type  Count  Volume (m3)  \
0    Double Leaf Metal Door - Metal Frame: D2 - 180...      1          NaN   
1      Metal Double Door Panel: D2 - 1800(W) x 2100(H)      1          NaN   
2       Painting to Door Frame: D2 - 1800(W) x 2100(H)      1          NaN   
3       Painting to Door Panel: D2 - 1800(W) x 2100(H)      1          NaN   
4        Ceiling - External Paint - Nippon Weatherbond      2         0.02   
..                                                 ...    ...          ...   
238                 Wall & Column - Internal Skim Coat      7         0.24   
239                       Wall - External Cement Board      1         0.01   
240  Wall - Wall & Panel - Block Wall - Aerated Aut...      4         4.82   
241  Louvre Glass Window 3 bays (1800 X 550): 3LW3 ...      1          NaN   
242  Louvre Glass Window 3 bays (1800 X 1500): 3LW2...      1          NaN   

     Area (m2) Length (mm)  Mass (kg)  Element  Model Level  ...  PSIS Code  \
0          NaN           6        NaN      NaN          NaN  ...  DRF10006H   
1         3.78         NaN        NaN      NaN          NaN  ...   DR10014D   
2          NaN           6        NaN      NaN          NaN  ...        NaN   
3         7.56         NaN        NaN      NaN          NaN  ...        NaN   
4         7.06    <varies>        NaN      NaN          NaN  ...        NaN   
..         ...         ...        ...      ...          ...  ...        ...   
238      78.86    <varies>        NaN      NaN          NaN  ...        NaN   
239       1.20        2.99        NaN      NaN          NaN  ...        NaN   
240      48.25    <varies>        NaN      NaN          NaN  ...        NaN   
241        NaN         NaN        NaN      NaN          NaN  ...   WF10010B   
242        NaN         NaN        NaN      NaN          NaN  ...   WF10010D   

                                      PSIS Description  WBS  \
0    METAL NON F/RATED DOOR FRAME 1800MM X 2100MM (...  NaN   
1                METAL DOOR LEAF (D/L) 1800MM X 2100MM  NaN   
2                                                  NaN  NaN   
3                                                  NaN  NaN   
4                                                  NaN  NaN   
..                                                 ...  ...   
238                                                NaN  NaN   
239                                                NaN  NaN   
240                                                NaN  NaN   
241  LOUVRE GLASS WINDOW WITH ALUMINIUM FRAME - 3 B...  NaN   
242  LOUVRE GLASS WINDOW WITH ALUMINIUM FRAME - 3 B...  NaN   

     Material: Railing Height (mm) Material: Frame Width (mm)  \
0                              NaN                        NaN   
1                              NaN                        NaN   
2                              NaN                        NaN   
3                              NaN                        NaN   
4                              NaN                        NaN   
..                             ...                        ...   
238                            NaN                        NaN   
239                            NaN                        NaN   
240                            NaN                        NaN   
241                            NaN                        NaN   
242                            NaN                        NaN   

     Material: Frame Profile  Material: Frame Thickness (mm)  \